<a href="https://colab.research.google.com/github/JunhanBai/Pratice/blob/main/Prompt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
REACT_TEMPLATE = '''\
You are a Python coding assistant. Use the ReAct pattern to solve the task.

Task: {task}

You can use these tools:
- PythonREPL(code): executes Python and returns stdout/stderr or exceptions
- Tests(code): runs predefined unit tests against `code` and returns a summary

Follow this format exactly:
Thought: reason step-by-step about the plan.
Action: <PythonREPL or Tests>
Action Input: <the code or nothing>
Observation: <the tool output>

(Repeat Thought/Action/Action Input/Observation as needed.)
When confident the code passes tests, finish with:
Final Answer: <ONLY the final Python code, no backticks>
'''

In [6]:
import sys, io, traceback

def PythonREPL(code: str):
    ns = {}
    out, err = io.StringIO(), io.StringIO()
    old_out, old_err = sys.stdout, sys.stderr
    try:
        sys.stdout, sys.stderr = out, err
        exec(compile(code, '<PythonREPL>', 'exec'), ns, ns)
        return {'ok': True, 'stdout': out.getvalue(), 'stderr': err.getvalue(), 'ns': ns}
    except Exception as e:
        tb = traceback.format_exc()
        return {'ok': False, 'error': str(e), 'traceback': tb, 'stdout': out.getvalue(), 'stderr': err.getvalue()}
    finally:
        sys.stdout, sys.stderr = old_out, old_err

def Tests(code: str, test_fn):
    ns = {}
    exec(compile(code, '<solution>', 'exec'), ns, ns)
    try:
        summary = test_fn(ns)
        return {'ok': True, 'summary': summary}
    except AssertionError as e:
        return {'ok': False, 'summary': str(e)}

In [7]:
from dataclasses import dataclass, field
from typing import List, Dict, Any, Callable

@dataclass
class TraceStep:
    thought: str
    action: str
    action_input: str
    observation: str

@dataclass
class ReActResult:
    final_code: str
    passed: bool
    test_summary: str
    trace: List[TraceStep] = field(default_factory=list)

def default_planner(task: str) -> str:
    task_l = task.lower()
    if 'top' in task_l and 'words' in task_l:
        return (
            "import re\n"
            "from collections import Counter\n\n"
            "def top_k_words(text, k):\n"
            "    tokens = re.findall(r\"[a-z0-9']+\", text.lower())\n"
            "    counts = Counter(tokens)\n"
            "    return [w for w,_ in counts.most_common(k)]\n"
        )
    # Fallback: fizzbuzz
    return (
        "def fizzbuzz(n):\n"
        "    out = []\n"
        "    for i in range(1, n+1):\n"
        "        if i % 15 == 0: out.append('FizzBuzz')\n"
        "        elif i % 3 == 0: out.append('Fizz')\n"
        "        elif i % 5 == 0: out.append('Buzz')\n"
        "        else: out.append(str(i))\n"
        "    return out\n"
    )

def patcher(original_code: str, feedback: str) -> str:
    # Simple example: no-op patcher, extend as needed based on feedback
    return original_code

def run_react(task: str, test_fn: Callable[[Dict[str, Any]], str]) -> ReActResult:
    trace: List[TraceStep] = []
    thought1 = 'Plan: generate initial solution, then run tests; patch if needed.'
    code1 = default_planner(task)
    obs1 = PythonREPL(code1)
    trace.append(TraceStep(thought1, 'PythonREPL', code1[:160], str(obs1)[:180]))
    obs2 = Tests(code1, test_fn)
    trace.append(TraceStep('Run tests on initial solution', 'Tests', '<solution>', str(obs2)))
    if obs2['ok']:
        return ReActResult(final_code=code1, passed=True, test_summary=str(obs2['summary']), trace=trace)
    thought3 = f"Tests failed; feedback: {obs2['summary']} — patch code and retry."
    code2 = patcher(code1, obs2['summary'])
    obs3 = PythonREPL(code2)
    trace.append(TraceStep(thought3, 'PythonREPL', code2[:160], str(obs3)[:180]))
    obs4 = Tests(code2, test_fn)
    trace.append(TraceStep('Re-run tests after patch', 'Tests', '<patched>', str(obs4)))
    return ReActResult(final_code=code2 if obs4['ok'] else code1,
                       passed=bool(obs4['ok']),
                       test_summary=str(obs4['summary']),
                       trace=trace)

In [8]:
TASK = (
    "Write a function `top_k_words(text, k)` that returns the top-k most frequent words. "
    "Ignore case and punctuation (treat `can't` as one token). If counts tie, any order is fine."
)

def tests_wordcount(ns):
    fn = ns.get('top_k_words')
    assert callable(fn), 'Function `top_k_words` not defined.'
    assert fn('A a a b b c', 2) == ['a', 'b']
    o = fn("Can't stop, won't stop! Can't STOP.", 2)
    assert set(o) == {"can't", 'stop'}, 'Should count words ignoring case and punctuation.'
    o2 = fn('Red fish, blue fish. Red fish!', 1)
    assert o2 in (['fish'], ['red']), 'Fish or Red can top; both valid.'
    return 'All tests passed.'

res = run_react(TASK, tests_wordcount)
print('Passed:', res.passed)
print('Test Summary:', res.test_summary)
for step in res.trace:
    print('\nThought:', step.thought)
    print('Action:', step.action)
    print('Observation:', step.observation[:200])

Passed: True
Test Summary: All tests passed.

Thought: Plan: generate initial solution, then run tests; patch if needed.
Action: PythonREPL
Observation: {'ok': True, 'stdout': '', 'stderr': '', 'ns': {'__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provid

Thought: Run tests on initial solution
Action: Tests
Observation: {'ok': True, 'summary': 'All tests passed.'}
